# 번역가는 대화에도 능하다

좋은 번역을 만드는 데에는 무슨 능력이 필요할까요? 

가장 먼저 떠오르는 것은 역시 언어 능력이죠! 적어도 번역하고자 하는 언어는 통달해야 좋은 번역을 해낼 수 있을 것 같습니다.  
하지만 뛰어난 언어 실력만으로 가능할까요?

말하고 싶은 것은, 번역가들의 번역이 단순히 언어를 변환하는 과정에 그치는 것이 아니라 원문을 이해하고 그 이해를 바탕으로 새로운 글을 작문하여 탄생한다는 겁니다.

인공지능도 마찬가지입니다. 

번역을 잘 해낼 수 있는 모델은 곧 언어를 잘 이해할 수 있는 모델이기도 해요.  
그래서 번역을 잘하는 트랜스포머가 자언어 이해(Natural Language Understanding) 모델의 근간이 되는 거죠!.  
질문과 답변을 주고받는 것 또한 제법 높은 수준의 자연어 이해를 요구하는데, 이것도 잘 해낼 수 있을지...

번역 모델을 활용한 챗봇 만들기! 시작!